# CEB 35300, Phylogenetic Comparative Methods 
## University of Chicago, 2018<br>Andrew Hipp, ahipp@mortonarb.org
### Session 4: Likelihood and information criteria

This tutorial is largely a chance for you to work through the material from today's lecture on likelihood and information criteria, implementing things to make sure you understand what you are doing. If you are interested in focusing instead on the parametric bootstrapping methods presented in Boettiger et al. 2012, I encourage you to work through his tutorial (in the `pmc` package, enter `vignette('pmc_tutorial'` to view the pdf). 

In either case, if you are working on your own data, as always I encourage you to try implementing these methods on your own dataset. It is most instructive to try to push your own data through a method, both because there are usually formatting difficulties to work through and because the results will be most sensible to you if you have prior expectations.

I will be circulating to address any questions. 

# Challenge questions relating to lnL and AIC

1. Write a function to take a single fitted model and return the AIC and AICc value. Use: 
    * ```logLik``` to extract the log likelihood from the fitted object --- you may assume for this exercise that you'll be using ```gls```, ```lm```, or some other model with a logLik method attached to it
    * ```attr(logLik(obj), 'df')``` to get the number of free model parameters, and 
    * ```attr(logLik(obj), 'nall')``` to get the sample size.
2. Write a function that takes a vector of AIC or AICc values and returns a vector of weights.
3. Write a wrapper function that calls both 1 and 2 above on a set of regression models, and returns model-averaged estimates for the model parameters.
4. Add model-averaged variances to your function # 3 using Burnham et al. 2011, formula at the bottom of pg. 26.
5. Calculate AICc weights for the OU models tested by Boettiger et al. 2012, Table 1. How well does AIC do at indicating best models and uncertainty about these compared to parametric bootstrapping? 
5. Simulate univariate data on a 100-taxon tree assuming &lambda; = 0.6, and compare Boettiger's parametric bootstrapping method for estimating the confidence about &lambda; with the 2 log-likelihood confidence interval. Use the ```pmc``` function in the ```pmc``` package. 

# ANSWERS -- NO PEEKING!!!!

### Challenge 1

In [8]:
aic <- function(x, ...) {
    dev <- -2 * logLik(x)
    K = attr(logLik(x), 'df')
    n = attr(logLik(x), 'nall')
    aic <- dev + 2*K
    aic.c <- aic + (2 * K * (K + 1)) / (n - K - 1)
    out <- list(aic = aic, aic.c = aic.c, n = n, K = K, lnL = logLik(x), param = as.numeric(x$modelStruct))
    class(out) <- 'aic'
    return(out)
}

### Challenge 2

In [9]:
aic.w <- function(x, ...) {
    aic.lnL <- exp(-0.5 * (x - min(x)))
    aic.w <- aic.lnL / sum(aic.lnL)
    return(aic.w)
}

### Challenge 3

In [10]:
aic.w.modelSet <- function(x, which.use = 'aic.c', ...) {
    aic.set <- lapply(x, aic)
    aic.set <- sapply(aic.set, function(x) x[[which.use]])
    out <- aic.w(aic.set, ...)
    return(out)
}

### TRYING IT OUT

library(phytools)
library(geiger)
library(nlme)
                      
tr <- pbtree(n = 100)
dat <- as.data.frame(sim.char(tr, matrix(c(1,0.8,
                                           0.8,1),2,2, byrow = T))[, , 1])
names(dat) <- c('y','x')
                      
models <- list(y.x.brown = gls(y ~ x, dat, correlation = corBrownian(1, tr)),
               y.x.pagel = gls(y ~ x, dat, correlation = corPagel(1, tr)), 
               y.brown = gls(y ~ 1, dat, correlation = corBrownian(1, tr)),
               y.x.star = gls(y ~ x, dat, correlation = corPagel(0, tr, fixed = TRUE)))

sapply(models, aic)    
round(aic.w.modelSet(models), 4)

,y.x.brown,y.x.pagel,y.brown,y.x.star
aic,153.857,154.9558,263.9908,331.4211
aic.c,154.107,155.3768,264.1145,331.6711
n,100,100,100,100
K,3,4,2,3
lnL,-73.9285,-73.47788,-129.9954,-162.7105
param,1,1.002416,1,0


y.x.brown y.x.pagel   y.brown  y.x.star 
   0.6536    0.3464    0.0000    0.0000

### Challenge 4
Check out the lecture online! 
http://systematics.mortonarb.org/lab/teaching/pcm35300-2018/WK04.MODELS/

### Challenge 5

In [16]:
K = c(2, 3, 5, 6, 17)
lnL = c(17.33, 15.69, 24.82, 26.69, 44.17)
aic.boett <- -2*lnL + 2*K
aicc.boett <- -2*lnL + 2*(K*(K + 1)) / (13 - K - 1) 
names(aic.boett) <- names(aicc.boett) <- c('BM', 'OU.1', 'OU.3', 'OU.4', 'OU.15')
message('AIC weights corresponding to Boettiger et al. 2012, Table 1')
cbind(aic.w = round(aic.w(aic.boett), 8),
      aic_c.w = round(aic.w(aicc.boett), 8)
     )


AIC weights corresponding to Boettiger et al. 2012, Table 1


,aic.w,aic_c.w
BM,0.00000719,0
OU.1,0.00000051,0
OU.3,0.00064119,0
OU.4,0.00153047,0
OU.15,0.99782063,1


Now try challenge 5, but without the OU.15 model... is this a model you would have tested in the first place? Are there reasonable ways to immunize oneself against introducing overly complex, biologically unreasonable models? or do some biologically reasonable models risk falling into the category of OU.15?